# 📓 LangChain Integration

TruLens provides TruChain, a deep integration with LangChain to allow you to
inspect and evaluate the internals of your application built using LangChain.
This is done through the instrumentation of key LangChain classes. To see a list
of classes instrumented, see *Appendix: Instrumented LangChain Classes and
Methods*.

In addition to the default instrumentation, TruChain exposes the
*select_context* method for evaluations that require access to retrieved
context. Exposing *select_context* bypasses the need to know the json structure
of your app ahead of time, and makes your evaluations re-usable across different
apps.

## Example Usage

Below is a quick example of usage. First, we'll create a standard LLMChain.

In [ ]:
# required imports
from langchain.chains import LLMChain
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate
from trulens_eval import TruChain

# typical langchain rag setup
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)
chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

To instrument an LLM chain, all that's required is to wrap it using TruChain.

In [ ]:
# instrument with TruChain
tru_recorder = TruChain(chain)

Similarly, LangChain apps defined with LangChain Expression Language (LCEL) are also supported.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

tru_recorder = TruChain(
    chain,
    app_id='Chain1_ChatApplication'
)

To properly evaluate LLM apps we often need to point our evaluation at an
internal step of our application, such as the retreived context. Doing so allows
us to evaluate for metrics including context relevance and groundedness.

For LangChain applications where the BaseRetriever is used, `select_context` can
be used to access the retrieved text for evaluation.

Example usage:

```python
context = TruChain.select_context(rag_chain)

f_context_relevance = (
    Feedback(provider.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
    )
```

For added flexibility, the select_context method is also made available through
`trulens_eval.app.App`. This allows you to switch between frameworks without
changing your context selector:

```python
from trulens_eval.app import App
context = App.select_context(rag_chain)
```

You can find the full quickstart available here: [LangChain Quickstart](../langchain_quickstart)

## Async Support

TruChain also provides async support for Langchain through the `acall` method. This allows you to track and evaluate async and streaming LangChain applications.

As an example, below is an LLM chain set up with an async callback.

In [ ]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models.openai import ChatOpenAI

from trulens_eval import TruChain

# Set up an async callback.
callback = AsyncIteratorCallbackHandler()

# Setup a simple question/answer chain with streaming ChatOpenAI.
prompt = PromptTemplate.from_template("Honestly answer this question: {question}.")
llm = ChatOpenAI(
    temperature=0.0,
    streaming=True, # important
    callbacks=[callback]
)
async_chain = LLMChain(llm=llm, prompt=prompt)

Once you have created the async LLM chain you can instrument it just as before.

In [ ]:
async_tc_recorder = TruChain(async_chain)

with async_tc_recorder as recording:
    await async_chain.acall(inputs=dict(question="What is 1+2? Explain your answer."))

For more usage examples, check out the [LangChain examples directory](https://github.com/truera/trulens/tree/main/trulens_eval/examples/expositional/frameworks/langchain).

## Appendix: Instrumented LangChain Classes and Methods

As of `trulens_eval` 0.20.0, TruChain instruments the following classes by default:

* `langchain_core.runnables.base.RunnableSerializable`
* `langchain_core.runnables.base.Serializable`
* `langchain_core.documents.base.Document`
* `langchain.chains.base.Chain`
* `langchain.vectorstores.base.BaseRetriever`
* `langchain_core.retrievers.BaseRetriever`
* `langchain_core.language_models.llms.BaseLLM`
* `langchain_core.language_models.base.BaseLanguageModel`
* `langchain_core.prompts.base.BasePromptTemplate`
* `langchain_core.memory.BaseMemory`
* `langchain_core.chat_history.BaseChatMessageHistory`
* `langchain.agents.agent.BaseSingleActionAgent`
* `langchain.agents.agent.BaseMultiActionAgent`
* `langchain_core.tools.BaseTool`

TruChain instruments the following methods:
* `invoke`
* `ainvoke`
* `save_context`
* `clear`
* `_call`
* `__call__`
* `_acall`
* `acall`
* `_get_relevant_documents`
* `_aget_relevant_documents`
* `get_relevant_documents`
* `aget_relevant_documents`
* `plan`
* `aplan`
* `_arun`
* `_run`

### Instrumenting other classes/methods.
Additional classes and methods can be instrumented by use of the
`trulens_eval.utils.instruments.Instrument` methods and decorators. Examples of
such usage can be found in the custom app used in the `custom_example.ipynb`
notebook which can be found in
`trulens_eval/examples/expositional/end2end_apps/custom_app/custom_app.py`. More
information about these decorators can be found in the
`trulens_instrumentation.ipynb` notebook.

### Inspecting instrumentation
The specific objects (of the above classes) and methods instrumented for a
particular app can be inspected using the `App.print_instrumented` as
exemplified in the next cell.

In [ ]:
async_tc_recorder.print_instrumented()